In [1]:
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [3]:
# Read CSV file into an RDD.
df1 = sc.textFile('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv')


#Read file into dataframe
df2=sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv',format='com.databricks.spark.csv',
                       header='true',inferSchema='true')
df2=sqlContext.read.format("com.databricks.spark.csv").options(header='true',inferSchema='true').load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv')

In [9]:
type(df1)

pyspark.rdd.RDD

In [20]:
type(df2)

pyspark.sql.dataframe.DataFrame

In [12]:
#RDD remove header and convert to dataframe
header=df1.first()

data=df1.filter(lambda x:x!=header)
data=data.map(lambda x:x.split(","))
data=data.map(lambda x:[float(i) for i in x])

In [13]:
df1=sqlContext.createDataFrame(data,header.split(","))

In [14]:
df1.printSchema()

root
 |-- number: double (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [15]:
df2.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [5]:
# Dataframe EDA
df2.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138097,3.1841611803868353,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287075,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.23551070057584,69.13785928889183,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.552813465531715,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.9535179651692,67.23801294602951,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.59820917078096,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574904,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.078778660148,0.0,17704.0


In [6]:
#df2=df2.na.drop(subset=['air_pressure_9am'])
df2=df2.drop('number')
df2=df2.na.drop()

In [24]:
df2.corr('air_pressure_9am','air_temp_9am')

-0.044446735577994535

In [25]:
sc.version

'1.6.0'

In [8]:
df2.columns

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

# Normal

In [11]:
binarizer=Binarizer(threshold=25.00,inputCol='relative_humidity_3pm',outputCol='label')
#threshold must be float, not int
binarydf=binarizer.transform(df2)

In [19]:
import pandas as pd
pd.DataFrame(binarydf.take(5),columns=binarydf.columns)

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm,label
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000,1.0
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597,0.0
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000,0.0
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547,0.0
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000,1.0


In [21]:
feature_columns=['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

assembler=VectorAssembler(inputCols=feature_columns,outputCol='feature')
assembledf=assembler.transform(binarydf)

In [24]:
pd.DataFrame(assembledf.take(5),columns=assembledf.columns)

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm,label,feature
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000,1.0,"[918.06, 74.822, 271.1, 2.0803542, 295.4, 2.86..."
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597,0.0,"[917.347688118, 71.4038426311, 101.935179356, ..."
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000,0.0,"[923.04, 60.638, 51.0, 17.0678522, 63.7, 22.10..."
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547,0.0,"[920.502751176, 70.1388948683, 198.832132717, ..."
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000,1.0,"[921.16, 44.294, 277.8, 1.8566602, 136.5, 2.86..."


In [27]:
(train,test)=assembledf.randomSplit([0.8,0.2],seed=613)

dt=DecisionTreeClassifier(labelCol='label',featuresCol='feature',maxDepth=5).fit(train)

pred=dt.transform(test)

In [31]:
pred.select('label',
 'rawPrediction',
 'probability',
 'prediction').take(5)

[Row(label=1.0, rawPrediction=DenseVector([3.0, 162.0]), probability=DenseVector([0.0182, 0.9818]), prediction=1.0),
 Row(label=1.0, rawPrediction=DenseVector([3.0, 162.0]), probability=DenseVector([0.0182, 0.9818]), prediction=1.0),
 Row(label=1.0, rawPrediction=DenseVector([3.0, 162.0]), probability=DenseVector([0.0182, 0.9818]), prediction=1.0),
 Row(label=1.0, rawPrediction=DenseVector([3.0, 162.0]), probability=DenseVector([0.0182, 0.9818]), prediction=1.0),
 Row(label=1.0, rawPrediction=DenseVector([3.0, 162.0]), probability=DenseVector([0.0182, 0.9818]), prediction=1.0)]

In [47]:
from pyspark.ml import evaluation
from pyspark.mllib.evaluation import BinaryClassificationMetrics,MulticlassMetrics

evaluator = evaluation.BinaryClassificationEvaluator(metricName='areaUnderROC') #areaUnderPR
evaluator.evaluate(pred)  

0.7904839546431482

In [48]:
predrdd=pred.select('label','prediction').rdd.map(lambda x:(x[0],x[1]))
metrics=MulticlassMetrics(predrdd)
# binary class only have areaunderroc and areaunderpr, multiclass have more metrics
metrics.precision()

In [50]:
dir(metrics)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_java_model',
 '_sc',
 'call',
 'confusionMatrix',
 'fMeasure',
 'falsePositiveRate',
 'precision',
 'recall',
 'truePositiveRate',
 'weightedFMeasure',
 'weightedFalsePositiveRate',
 'weightedPrecision',
 'weightedRecall',
 'weightedTruePositiveRate']

In [52]:
metrics.confusionMatrix().toArray()

array([[ 117.,   30.],
       [  14.,   73.]])

# Pipelline

In [53]:
stage=[]

binarizer=Binarizer(threshold=25.00,inputCol='relative_humidity_3pm',outputCol='label')
stage=stage+[binarizer]

feature_columns=['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

assembler=VectorAssembler(inputCols=feature_columns,outputCol='feature')
stage=stage+[assembler]

model=DecisionTreeClassifier(labelCol='label',featuresCol='feature',maxDepth=5)
stage=stage+[model]


In [55]:
(train,test)=df2.randomSplit([0.8,0.2],seed=613)

pipeline=Pipeline(stages=stage)

dt2=pipeline.fit(train)
pred2=dt2.transform(test)

In [57]:
evaluator = evaluation.BinaryClassificationEvaluator(metricName='areaUnderROC') #areaUnderPR
evaluator.evaluate(pred2)

0.7904839546431482